# Preparação Dos Dados


In [11]:
import pandas as pd
import numpy as np

file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Define os intervalos de defasagem a serem testados
intervalos = [15, 30, 45]

# Loop sobre cada intervalo de defasagem
for intervalo in intervalos:
    
    for i in range(intervalo, 0, -1):
        data[f'Entrada {i}'] = data['Vazao'].shift(i)

    # Define a ordem das novas colunas (defasagens de entrada + coluna original 'Vazao')
    novas_colunas = [f'Entrada {i}' for i in range(intervalo, 0, -1)] + ['Vazao']
    data = data[novas_colunas]

    # Criação das colunas de saída defasadas
    for i in range(1, 12):
        data[f'Saida {i}'] = data['Vazao'].shift(-i)

    # escolhi  exibir a primeira e a última coluna
    subset_columns =  [data.columns[0], 'Vazao', data.columns[-1]]
    
    print(f"\nDataframe para {intervalo} entradas:")
    print(data.loc[:, subset_columns]) # Exibe as primeiras 2 linhas do DataFrame para verificar o resultado



Dataframe para 15 entradas:
       Entrada 15  Vazao  Saida 11
0             NaN   2110    2000.0
1             NaN   2089    2000.0
2             NaN   2050    1981.0
3             NaN   2010    1990.0
4             NaN   1991    1991.0
...           ...    ...       ...
17284      3790.0   5418       NaN
17285      3909.0   5275       NaN
17286      3992.0   5055       NaN
17287      4141.0   4818       NaN
17288      4307.0   4596       NaN

[17289 rows x 3 columns]

Dataframe para 30 entradas:
       Entrada 30  Vazao  Saida 11
0             NaN   2110    2000.0
1             NaN   2089    2000.0
2             NaN   2050    1981.0
3             NaN   2010    1990.0
4             NaN   1991    1991.0
...           ...    ...       ...
17284      3175.0   5418       NaN
17285      3160.0   5275       NaN
17286      3188.0   5055       NaN
17287      3202.0   4818       NaN
17288      3141.0   4596       NaN

[17289 rows x 3 columns]

Dataframe para 45 entradas:
       Entrada 45  Va

# Segunda preparação



In [112]:
import pandas as pd
import numpy as np
import sys

# Carregar o arquivo Excel
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Adicionar colunas V1 e V2
data['V1'] = np.nan
data['V2'] = np.nan

# Calcular V1 baseado na coluna Vazao
data.loc[1:, 'V1'] = ((data['Vazao'].iloc[1:] - data['Vazao'].shift(1).iloc[1:]) / data['Vazao'].shift(1).iloc[1:]) * 100

# Calcular V2 baseado na coluna V1
data.loc[1:, 'V2'] = data['V1'].iloc[1:] - data['V1'].shift(1).iloc[1:]


# Define os intervalos de defasagem a serem testados
intervalos = [15, 30, 45]

# Define os intervalos de defasagem a serem testados
coluna = input("Digite a coluna desejada para fazer a defasagem (V1, V2 ou Vazao)").capitalize()

if(coluna not in data.columns):
    print("Coluna inexistente")
    sys.exit()

# Loop sobre cada intervalo de defasagem
for intervalo in intervalos:
    for i in range(intervalo, 0, -1):
        data[f'Entrada {i}'] = data[coluna].shift(i)

    # Define a ordem das novas colunas (defasagens de entrada + coluna original 'Vazao')
    novas_colunas = [f'Entrada {i}' for i in range(intervalo, 0, -1)] + [coluna]
    data = data[novas_colunas]

    # Criação das colunas de saída defasadas
    for i in range(1, 12):
        data[f'Saida {i}'] = data[coluna].shift(-i)

    # escolhi  exibir a primeira e a última coluna
    subset_columns =  [data.columns[0], coluna, data.columns[-1]]
    
    print(f"\nDataframe para {intervalo} entradas:")
    print(data.loc[:, subset_columns]) # Exibe as primeiras 2 linhas do DataFrame para verificar o resultado  


Dataframe para 15 entradas:
       Entrada 15        V1  Saida 11
0             NaN       NaN  0.000000
1             NaN -0.995261  0.000000
2             NaN -1.866922 -0.950000
3             NaN -1.951220  0.454316
4             NaN -0.945274  0.050251
...           ...       ...       ...
17284    3.978052  0.389105       NaN
17285    3.139842 -2.639350       NaN
17286    2.123305 -4.170616       NaN
17287    3.732465 -4.688427       NaN
17288    4.008694 -4.607721       NaN

[17289 rows x 3 columns]

Dataframe para 30 entradas:
       Entrada 30        V1  Saida 11
0             NaN       NaN  0.000000
1             NaN -0.995261  0.000000
2             NaN -1.866922 -0.950000
3             NaN -1.951220  0.454316
4             NaN -0.945274  0.050251
...           ...       ...       ...
17284   -2.994195  0.389105       NaN
17285   -0.472441 -2.639350       NaN
17286    0.886076 -4.170616       NaN
17287    0.439147 -4.688427       NaN
17288   -1.905059 -4.607721       NaN

[17